In [52]:
%matplotlib widget

import rosbag
import matplotlib.pyplot as plt
import numpy as np
from math import *
from scipy.interpolate import interp1d
from transformations import * 
plt.rc('figure', figsize=(10,8))
#plt.rc('figure', figsize=(20,15))

def quat2eulers(w, x, y ,z):
    r = atan2(2 * (w * x + y * z),
                    1 - 2 * (x * x + y * y))
    p = asin(2 * (w * y - z * x))
    y = atan2(2 * (w * z + x * y), 1 - 2 * (y * y + z * z))
    return y, p, r

def read_pose(bag, topic):
    pos = []
    ypr = []
    ts = []
    print(f"Read poses from topic {topic}")
    for topic, msg, t in bag.read_messages(topics=[topic]):
        p = msg.pose.position
        q = msg.pose.orientation
        pos.append([p.x, p.y, p.z])
        y, p, r = quat2eulers(q.w, q.x, q.y, q.z)
        ypr.append([y, p, r])
        ts.append(msg.header.stamp.to_sec())
    ret = {
        "t": np.array(ts),
        "pos": np.array(pos),
       "pos_func": interp1d(ts, pos,axis=0),
        "ypr": np.array(ypr),
        "ypr_func": interp1d(ts, ypr,axis=0)
    }
    return ret

def read_loops(bag, topic="/swarm_loop/loop_connection"):
    loops = []
    for topic, msg, t in bag.read_messages(topics=[topic]):
        loop = {
            "ts_a": msg.ts_a.to_sec(),
            "ts_b": msg.ts_b.to_sec(),
            "id_a":msg.id_a,
            "id_b":msg.id_b,
            "dpos":np.array([msg.dpos.x, msg.dpos.y, msg.dpos.z]),
            "dyaw":msg.dyaw
        }
        loops.append(loop)
    return loops
    
def bag_read(bagname, nodes = [1, 2]):
    bag = rosbag.Bag(bagname)
    poses = {}
    loops = read_loops(bag, "/swarm_loop/loop_connection")
    for i in nodes:
        poses[i] =  read_pose(bag, f"/SwarmNode{i}/pose")
    bag.close()
    return poses, loops
    


In [56]:
nodes = [1, 2]
#poses, loops =bag_read("data/swarm_local_2020-12-05-23-31-24-flyzone.bag", nodes) ##This bag has out of flyzone part
poses, loops = bag_read("data/swarm_local_2020-12-07-17-09-41.bag", nodes)
#poses, loops = bag_read("data/swarm_local_2020-12-07-17-34-57.bag", nodes)


Read poses from topic /SwarmNode1/pose
Read poses from topic /SwarmNode2/pose


In [57]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax = fig.gca(projection='3d')
for i in nodes:
    ax.plot(poses[i]["pos"][:,0], poses[i]["pos"][:,1],poses[i]["pos"][:,2], label=f"ViconPOS{i}")

quivers = []
for loop in loops:
    posa_gt = poses[loop["id_a"]]["pos_func"](loop["ts_a"])
    posb_gt = poses[loop["id_b"]]["pos_func"](loop["ts_b"])
    quivers.append([posa_gt[0], posa_gt[1], posa_gt[2], posb_gt[0]-posa_gt[0], posb_gt[1]-posa_gt[1], posb_gt[2]-posa_gt[2]])
quivers = np.array(quivers)
c = np.arctan2(quivers[:,4], quivers[:,3])
c = (c.ravel() - c.min()) / c.ptp()
c = np.concatenate((c, np.repeat(c, 2)))
c = plt.cm.hsv(c)
ax.quiver(quivers[:,0], quivers[:,1], quivers[:,2], quivers[:,3], quivers[:,4], quivers[:,5], arrow_length_ratio=0.1, colors = c)

plt.legend()
plt.show()

plt.figure()
for i in nodes:
    plt.plot(poses[i]["ypr"][:,0], label=f"ViconYaw{i}")
    plt.plot(poses[i]["ypr"][:,0], label=f"ViconPitch{i}")
    plt.plot(poses[i]["ypr"][:,0], label=f"ViconRoll{i}")
    plt.legend()
    plt.grid()
    plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
_loops_data = []
dpos_loops = []
dpos_gts = []
dpos_gt_norms= []
dpos_loop_norms= []
dpos_errs = []
dpos_errs_norm = []
posa_gts = []
ts_a = []
dyaws = []
dyaw_gts = []
yawa_gts = []
yawb_gts = []
dyaw_errs = []
print("Total loops", len(loops))
for loop in loops:
    posa_gt = poses[loop["id_a"]]["pos_func"](loop["ts_a"])
    posb_gt = poses[loop["id_b"]]["pos_func"](loop["ts_b"])
    yawa_gt = poses[loop["id_a"]]["ypr_func"](loop["ts_a"])[0]
    yawb_gt = poses[loop["id_b"]]["ypr_func"](loop["ts_b"])[0]
    dyaw_gt = yawb_gt - yawa_gt
    dpos_gt = np.array(posb_gt - posa_gt)
    dpos_gt = np.transpose([dpos_gt])
    Re = rotation_matrix(-yawa_gt, [0, 0, 1])[0:3, 0:3]
    dpos_gt = np.dot(Re, dpos_gt)
    dpos_gt = dpos_gt.flatten()
    dpos_loop = np.array(loop["dpos"])
    _loops_data.append({
        "dpos_loop": dpos_loop,
        "dpos_gt": dpos_gt,
        "dpos_err": dpos_gt - dpos_loop
        })
    dpos_loops.append(dpos_loop)
    dpos_gts.append(dpos_gt)
    dpos_errs.append(dpos_gt - dpos_loop)    
    dpos_gt_norms.append(np.linalg.norm(dpos_gt))
    dpos_loop_norms.append(np.linalg.norm(dpos_loop))
    dpos_errs_norm.append(np.linalg.norm(dpos_gt - dpos_loop))
    posa_gts.append(posa_gt)
    dyaws.append(loop["dyaw"])
    dyaw_gts.append(yawb_gt-yawa_gt)
    ts_a.append(loop["ts_a"])
    yawa_gts.append(yawa_gt)
    yawb_gts.append(yawb_gt)
    dyaw_errs.append(yawb_gt-yawa_gt-loop["dyaw"])
    
posa_gts = np.array(posa_gts)
dpos_errs = np.array(dpos_errs)
fig = plt.figure()

plt.subplot(311)
#plt.plot(ts_a, dpos_errs_norm, '.', label="Loop Error")
#plt.plot(ts_a, np.abs(dpos_errs[:,0]), '+', label="Loop Error X")
#plt.plot(ts_a, np.abs(dpos_errs[:,1]), '+', label="Loop Error Y")
plt.plot(ts_a, dpos_errs[:,2], '+', label="Loop Error Z")
plt.title("Error Pos Loop vs Vicon")
plt.grid(which="both")
plt.legend()

plt.subplot(312)
plt.plot(ts_a, dyaws, '.', label="DYaw Gt")
plt.plot(ts_a, dyaw_gts, '+', label="DYaw Loop")
plt.plot(ts_a, yawa_gts, "*", label="Vicon Yaw A")
plt.plot(ts_a, np.abs(dyaw_errs), "x", label="DYaw Error")

#plt.plot(ts_a, yawb_gts, "*", label="Vicon Yaw B")
plt.title("Error Yaw Loop vs Vicon")
plt.grid(which="both")
plt.legend()


plt.subplot(313)
plt.plot(ts_a, posa_gts[:,0], '+', label="Vicon X")
plt.plot(ts_a, posa_gts[:,1], '+', label="Vicon Y")
plt.plot(ts_a, posa_gts[:,2], '+', label="Vicon Z")
plt.plot(poses[i]["t"], poses[i]["pos"][:,0], label="Vicon X")
plt.plot(poses[i]["t"], poses[i]["pos"][:,1], label="Vicon Y")
plt.plot(poses[i]["t"], poses[i]["pos"][:,2], label="Vicon Z")

plt.grid(which="both")
plt.legend()
plt.show()

plt.figure()
plt.subplot(141)
plt.hist(dpos_errs_norm, 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(142)
plt.hist(dpos_errs[:,0], 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(143)
plt.hist(dpos_errs[:,1], 5, density=True, facecolor='g', alpha=0.75)
plt.subplot(144)
plt.hist(dpos_errs[:,2], 5, density=True, facecolor='g', alpha=0.75)


print("Pos cov", np.cov(dpos_errs[:,0]), np.cov(dpos_errs[:,1]), np.cov(dpos_errs[:,2]) )
print("Yaw cov", np.cov(dyaw_errs))
plt.show()


# plt.figure()
# plt.subplot(211)
# plt.plot(dpos_gt_norms, dpos_errs_norm, 'o', label="GtDistance vs Error")
# plt.grid(which="both")
# plt.subplot(212)
# plt.plot(dpos_loop_norms, dpos_errs_norm, 'o', label="LoopDistance vs Error")
# plt.grid(which="both")
# plt.legend()
# plt.show()

Total loops 45


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pos cov 0.1352192034497826 0.739531577070131 0.023797876218099923
Yaw cov 0.05572986261394686
